In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Cargar archivos desde la carpeta exportada
rutas = pd.read_csv("exported_data/rutas.csv")
bases = pd.read_csv("exported_data/bases.csv")
obstaculos = pd.read_csv("exported_data/obstaculos.csv")
urgencias = pd.read_csv("exported_data/urgencias.csv")

# Dimensiones del grid
max_row = max(rutas['row'].max(), bases['row'].max(), obstaculos['row'].max(), urgencias['row'].max()) + 1
max_col = max(rutas['col'].max(), bases['col'].max(), obstaculos['col'].max(), urgencias['col'].max()) + 1

# Ticks y drones
ticks = rutas['tick'].max() + 1
num_drones = rutas['dron'].nunique()

# Crear figura
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(-0.5, max_col - 0.5)
ax.set_ylim(max_row - 0.5, -0.5)

# Grid que marca las celdas (líneas entre celdas)
ax.set_xticks([x - 0.5 for x in range(max_col + 1)], minor=False)
ax.set_yticks([y - 0.5 for y in range(max_row + 1)], minor=False)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True, which='major', color='gray', linewidth=0.5)

# Artistas - ahora sin sumar 0.5
colors = plt.cm.tab10(range(num_drones))
drone_plots = [ax.plot([], [], 'o', color=colors[i], markersize=10, label=f'Dron {i}')[0] for i in range(num_drones)]

# Obstáculos, bases y urgencias en el centro de las celdas (sin sumar 0.5)
ax.scatter(obstaculos['col'], obstaculos['row'], marker='X', color='black', s=100, label='Obstáculo', zorder=3)
ax.scatter(bases['col'], bases['row'], marker='s', color='blue', s=100, label='Base', zorder=3)

# Urgencia como fondo tipo heatmap
urgencia_grid = [[0 for _ in range(max_col)] for _ in range(max_row)]
for _, row in urgencias.iterrows():
    urgencia_grid[row['row']][row['col']] = row['peso']

ax.imshow(urgencia_grid, cmap='Reds', origin='lower', 
          extent=(-0.5, max_col - 0.5, -0.5, max_row - 0.5), 
          alpha=0.4, zorder=1)

# Función de actualización
def update(tick):
    data_tick = rutas[rutas['tick'] == tick]
    for i, plot in enumerate(drone_plots):
        dron_data = data_tick[data_tick['dron'] == i]
        if not dron_data.empty:
            # Sin sumar 0.5 - los drones van por el centro de las celdas
            plot.set_data(dron_data['col'], dron_data['row'])
    ax.set_title(f"Tick {tick}")
    return drone_plots

ani = animation.FuncAnimation(fig, update, frames=ticks, interval=300, blit=True)
plt.legend(loc='upper right')
plt.close()
HTML(ani.to_jshtml())